Quelle est la précision du modèle Random Forest par rapport aux arbres de décision simple pour la prédiction de l'approbation des prêts?

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Charger les données (Remplacer 'loan_data.csv' par votre fichier de données)
data = pd.read_excel('train (1).xlsx')
# Affichage des types de colonnes pour identifier celles de type 'object'
print(data.dtypes)

Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount             int64
Loan_Amount_Term       int64
Credit_History         int64
Property_Area         object
Loan_Status           object
dtype: object


In [28]:
# Fonction pour détecter et convertir les types mixtes
def clean_column(column):
    # Essayer de convertir en numérique, sinon laisser en chaîne de caractères
    try:
        return pd.to_numeric(column, errors='coerce')
    except:
        return column.astype(str)

In [29]:
# Appliquer la fonction de nettoyage sur toutes les colonnes de type 'object'
for column in data.select_dtypes(include=['object']).columns:
    data[column] = clean_column(data[column])

In [30]:
# Afficher les types de colonnes après nettoyage
print(data.dtypes)

Loan_ID              float64
Gender               float64
Married              float64
Dependents           float64
Education            float64
Self_Employed        float64
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount             int64
Loan_Amount_Term       int64
Credit_History         int64
Property_Area        float64
Loan_Status          float64
dtype: object


In [31]:
# Colonnes catégorielles à encoder
categorical_columns = ['Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area']

In [32]:
# Encoder les colonnes catégorielles
label_encoders = {}
for column in categorical_columns:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

In [9]:
# Séparation des features et de la cible
X = data.drop('Loan_Status', axis=1)  # Remplacer 'Loan_Status' par le nom de la colonne cible
y = data['Loan_Status']

In [10]:
# Encoder la colonne cible
le_target = LabelEncoder()
y = le_target.fit_transform(y)

In [12]:
# Séparer les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# Entraîner un modèle d'arbre de décision
decision_tree = DecisionTreeClassifier(random_state=42)
decision_tree.fit(X_train, y_train)
y_pred_tree = decision_tree.predict(X_test)
accuracy_tree = accuracy_score(y_test, y_pred_tree)


In [20]:
from sklearn.impute import SimpleImputer

# Supprimer la colonne 'Loan_ID'
X_train = X_train.drop(columns=['Loan_ID'])
X_test = X_test.drop(columns=['Loan_ID'])

# Imputer les valeurs manquantes avec la moyenne
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

In [22]:
# Vérifier les valeurs manquantes dans X_train
missing_values = X_train.isnull().sum()
print("Nombre de valeurs manquantes par colonne dans X_train :\n", missing_values)

# Vérifier les valeurs manquantes dans X_test
missing_values_test = X_test.isnull().sum()
print("Nombre de valeurs manquantes par colonne dans X_test :\n", missing_values_test)

Nombre de valeurs manquantes par colonne dans X_train :
 Gender                0
Married               0
Dependents           36
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term      0
Credit_History        0
Property_Area         0
dtype: int64
Nombre de valeurs manquantes par colonne dans X_test :
 Gender               0
Married              0
Dependents           7
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
dtype: int64


In [23]:


# Imputer les valeurs manquantes dans la colonne 'Dependents'
imputer = SimpleImputer(strategy='most_frequent')  # Utilisation de la stratégie la plus fréquente pour les variables catégorielles

# Imputation sur X_train
X_train['Dependents'] = imputer.fit_transform(X_train[['Dependents']])

# Imputation sur X_test
X_test['Dependents'] = imputer.transform(X_test[['Dependents']])

# Vérifier à nouveau les valeurs manquantes pour s'assurer que tout est imputé
print("Nombre de valeurs manquantes par colonne dans X_train après imputation :\n", X_train.isnull().sum())
print("Nombre de valeurs manquantes par colonne dans X_test après imputation :\n", X_test.isnull().sum())


Nombre de valeurs manquantes par colonne dans X_train après imputation :
 Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
dtype: int64
Nombre de valeurs manquantes par colonne dans X_test après imputation :
 Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
dtype: int64


In [24]:
# Entraîner un modèle Random Forest
random_forest = RandomForestClassifier(random_state=42)
random_forest.fit(X_train, y_train)
y_pred_forest = random_forest.predict(X_test)
accuracy_forest = accuracy_score(y_test, y_pred_forest)

In [25]:
# Afficher les résultats
print(f"Précision de l'arbre de décision: {accuracy_tree:.2f}")
print(f"Précision du Random Forest: {accuracy_forest:.2f}")


Précision de l'arbre de décision: 1.00
Précision du Random Forest: 1.00


# Interprétation
Ces résultats indiquent que les modèles d'arbre de décision et de forêt aléatoire ont atteint une précision de 100 % lors de la prédiction du statut d'approbation du prêt sur les données d'entraînement et de test. Cela signifie que les modèles ont réussi à classer correctement chaque exemple dans l'ensemble de données en fonction de leur statut d'approbation du prêt qu'il s'agisse de "Approuvé" ou "Refusé", en utilisant les variables disponibles telles que le genre, l'état matrimonial, le revenu de l'emprunteur, le montant du prêt, etc. Cependant, une précision de 100 % sur les données de test peut également indiquer un surajustement du modèle aux données d'entraînement.

In [34]:
# Extraire un arbre de la forêt
clf_tree = random_forest.estimators_[0]

In [26]:
# Comparer les deux modèles
if accuracy_forest > accuracy_tree:
    print("Le modèle Random Forest a une meilleure précision que l'arbre de décision.")
else:
    print("L'arbre de décision a une meilleure précision que le modèle Random Forest.")

L'arbre de décision a une meilleure précision que le modèle Random Forest.


# Interprétation
Sur l'ensemble de données de test l'arbre de décision a produit des prédictions plus précises que le modèle Random Forest. En d'autres termes, lorsqu'il s'agit de prédire le statut d'approbation du prêt pour de nouveaux exemples, l'arbre de décision a mieux généralisé et effectué des prédictions plus fiables par rapport au modèle Random Forest.